<a href="https://colab.research.google.com/github/farrukh602/AI_Math/blob/main/Copy_of_building_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install crewai
!pip install langchain
!pip install openai
# !pip install crewai_tools
# !pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.7/194.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 774.0/774.0 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.6/327.6 kB 37.6 MB/s eta 0:00:00
  Attempting uninstall: regex
    Found existing installation: regex 2024.5.15
    Uninstalling regex-2024.5.15:
      Successfully uninstalled regex-2024.5.15
  Attempting uninstall: embedchain
    Found existing installation: embedchain 0.1.112
    Uninstalling embedchain-0.1.112:
      Successfully uninstalled embedchain-0.1.112


In [ ]:
import crewai
from crewai import Agent, Task, Crew
import os
from openai import OpenAI
# Set the API key
os.environ["OPENAI_API_KEY"] = ....................

# Initialize the OpenAI client (if needed)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Multi Agents
Six key elements which help agents perform better:
- Role playing
- Focus
- Tools
- Cooperation
- Guiderails
- Memory

### **Role Playing, Focus, and Cooperation**

In [ ]:
# Defining Agents
researcher = Agent(role='Senior Researcher',
                   goal = "Be the most knowledgeable and resourceful researcher "
         "in your team, focusing on the latest advancements in "
         "computational fluid dynamics (CFD) and mechanical design.",
                   backstory=("You are a Senior Researcher with a deep passion for "
        "mechanical engineering and CFD. You work at a top-tier "
        "engineering firm, collaborating with a team of experts "
        "to push the boundaries of design and analysis."
        "Your primary objective is to stay at the forefront of "
        "technological advancements in CFD and mechanical design, "
        "providing comprehensive and detailed research findings "
        "to support your team."
        "Make sure to provide full complete answers, "
        "and make no assumptions."),
                   allow_delegation=False,
                   verbose=True
                   )
support_quality_assurance_agent = Agent(role='Research Quality Assurance Specialist',
                                        goal="Get recognition for ensuring the highest quality and accuracy "
         "in research outputs within your team.",
                                        backstory=( "You are a Research Quality Assurance Specialist with a keen eye "
        "for detail and a strong commitment to excellence. "
        "You work at a leading engineering firm, where your primary responsibility "
        "is to ensure that all research conducted by your team meets the highest standards "
        "of accuracy and completeness.\n"
        "You are currently collaborating with your team on a project focusing on {project_topic}, "
        "ensuring that the research conducted is thorough, accurate, and assumes nothing."
        "Your goal is to make sure that the research reports provided by the team are complete, "
        "precise, and actionable."),
                                        allow_delegation=True,
                                        verbose=True)

#### Points to consider
- Role Playing: both agents have been given role, goal and backstory
- Focus: Both agents have been prompted to get into the character of roles they are playing
- Cooperation: Support quality assurance agent can delegate work back to Senior Researcher allowing these agents to work together.

### **Tools, Guardrails, and Memory**

#### *Tools*

In [ ]:
from crewai_tools import SerperDevTool, \
                        ScrapeWebsiteTool, \
                        WebsiteSearchTool

##### Different Ways to Give Agents Tools

- Agent Level: The Agent can use the Tool(s) on any Task it performs.
- Task Level: The Agent will only use the Tool(s) when performing that specific Task.

**Note**: Task Tools override the Agent Tools.

In [ ]:
# instantiate the tools
search_tool=SerperDevTool()
docs_scrape_tool1 = ScrapeWebsiteTool(
    website_url="https://www.neuralconcept.com/post/applying-machine-learning-in-cfd-to-accelerate-simulation#:~:text=Artificial%20Intelligence%20can%20boost%20CFD,and%20heat%20transfer%20in%20common."
)

docs_scrape_tool2 = ScrapeWebsiteTool(
    website_url="https://en.wikipedia.org/wiki/Computational_fluid_dynamics#:~:text=Computational%20fluid%20dynamics%20(CFD)%20is,problems%20that%20involve%20fluid%20flows."
)


In [ ]:
# Defining Tasks
ai_application_in_cfd = Task(
    description=("Conduct comprehensive online research on {project_topic_ai}.\n\n"
        "Explore AI-driven optimization techniques, machine learning models "
        "for fluid flow prediction, and advancements in automated mesh generation."
    ),
    expected_output = ("A detailed research summary highlighting key findings on AI applications "
        "in CFD, including examples of successful implementations and their impact "
        "on computational efficiency and accuracy.\n"
        "Provide references to academic papers, industry reports, and software tools."
),
    tools=[search_tool, docs_scrape_tool1],
    agent=researcher,
    verbose=True
)

cfd_advancements_research = Task(
    description = ("Conduct comprehensive research on the latest advancements on {project_topic_cfd}"),
    expected_output = ("A detailed research report summarizing recent advancements in CFD "
        "for aerospace applications.\n"
        "Include references to academic papers, conference proceedings, and "
        "industry reports supporting the findings."),
    tools = [search_tool, docs_scrape_tool2],
    agent=researcher,
    verbose=True
)


# Task: Verify AI Applications Research in Mechanical Design
ai_applications_quality_assurance = Task(
    description=(
        "Verify the accuracy and completeness of the research on AI applications "
        "in optimizing mechanical design processes conducted by the Senior Researcher.\n\n"
        "Ensure that the report includes comprehensive references and supports "
        "all claims with relevant data and examples."
    ),
    expected_output=(
        "A quality assurance assessment confirming the reliability and relevance "
        "of the AI applications research in mechanical design.\n"
        "Provide recommendations for enhancing the report's clarity and "
        "expanding on practical implications."
    ),
    agent=support_quality_assurance_agent,
)



cfd_advancements_quality_assurance = Task(
    description=(
        "Review the research report on recent advancements in computational "
        "fluid dynamics (CFD) for aerospace applications produced by the Senior Researcher.\n\n"
        "Ensure the report meets high standards of accuracy, clarity, and relevance to "
        "current industry challenges."
    ),
    expected_output=(
        "A detailed quality assurance report assessing the strengths and weaknesses "
        "of the CFD advancements research.\n"
        "Provide constructive feedback on clarity, completeness, and relevance of "
        "the findings."
    ),
    agent=support_quality_assurance_agent,
)

### Creating the Crew

#### Memory
- Setting `memory=True` when putting the crew together enables Memory.

In [ ]:
crew = Crew(
    agents = [researcher, support_quality_assurance_agent],
    tasks = [ai_application_in_cfd, cfd_advancements_research, cfd_advancements_quality_assurance],
    memory=True,
)

### Running the Crew
#### Guardrails
- By running the execution below, you can see that the agents and the responses are within the scope of what we expect from them.

In [ ]:
# Define the inputs for the Senior Researcher (researcher)

inputs = {
    'project_topic': "AI Applications in Mechanical Design and CFD and recent advancements in CFD",
    'project_topic_cfd': "Recent Advancements in Computational Fluid Dynamics (CFD)",
    'project_topic_ai': "AI Applications in Mechanical Design and CFD"

}

# Kick off the crew with the provided inputs
result = crew.kickoff(inputs=inputs)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}